<a href="https://colab.research.google.com/github/amokhtare-ivsonance/AcousTools/blob/main/StitchedTorous_ls_Yump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy gdstk sympy scipy if running in a notebook environment
import numpy as np
import sympy as sp
import gdstk
import math

# Use !pip install numpy gdstk sympy scipy if running in a notebook environment

def run_design():
    # Loop for specific designs
    for designs in [4]:

        # --- Constants & Setup ---
        cglass = 3280.0  # m/s sound speed in glass
        cWater = 1523.0  # m/s sound speed in water
        m = 1            # Order of the vortex
        l = 1
        s = designs
        GapJumps = 0

        # Frequency arrays
        freqhl1 = np.arange(44.0e6, 44.0e6 - 0.01e6, -0.01e6)

        if len(freqhl1) == 0:
            freqhl1 = np.array([44.0e6])

        freqhl = freqhl1
        nc = 1
        freq = np.tile(freqhl, nc)

        lambda_val = cglass / freq
        kl = 2 * np.pi / lambda_val

        zfix = 1100e-6 + 0.0e-6
        zfixbar = kl * zfix

        Fn = 12
        FFn = 1

        freq_mhz = freq[0] / 1e6

        strv = f"StitchedTorousJump-{l}{s}-Freq-{freq_mhz}-MHz-Cont{Fn}"
        DegreeAngle = 0 * np.pi / 8

        # Initial Angle Calculation
        val = (zfixbar[0]/m + np.pi) / (2*np.pi)
        iniang_scalar = 2 * np.pi * np.floor(val) + 2 * np.pi + DegreeAngle
        iniangj_scalar = 2 * np.pi * np.floor(val) + 2 * np.pi

        iniang = iniang_scalar * np.ones_like(zfixbar)

        discth = 100
        angth = 2 * np.pi / discth

        # --- Symbolic Definitions ---
        theta_sym, thetac_sym, thick_sym, z_sym = sp.symbols('theta thetac thick z')

        # Define rho equations symbolically
        rho1_sym = (1/kl[0]) * sp.sqrt((m*(theta_sym) + l*thetac_sym)**2 - z_sym**2)
        rho2_sym = (1/kl[0]) * sp.sqrt((m*(theta_sym - sp.pi) + l*(thetac_sym))**2 - z_sym**2)

        # Coordinate transformations
        X1_sym = rho2_sym * sp.cos(theta_sym)
        Y1_sym = rho2_sym * sp.sin(theta_sym)
        X2_sym = rho1_sym * sp.cos(theta_sym)
        Y2_sym = rho1_sym * sp.sin(theta_sym)

        # Normals (Derivatives)
        dX1 = sp.diff(X1_sym, theta_sym)
        dY1 = sp.diff(Y1_sym, theta_sym)
        denom1 = sp.sqrt(dX1**2 + dY1**2)
        Nx1_sym = -dY1 / denom1
        Ny1_sym = dX1 / denom1

        dX2 = sp.diff(X2_sym, theta_sym)
        dY2 = sp.diff(Y2_sym, theta_sym)
        denom2 = sp.sqrt(dX2**2 + dY2**2)
        Nx2_sym = -dY2 / denom2
        Ny2_sym = dX2 / denom2

        # Upper/Lower boundaries
        x1lower_sym = X1_sym + Nx1_sym * thick_sym / 4
        y1lower_sym = Y1_sym + Ny1_sym * thick_sym / 4
        x1upper_sym = X1_sym - Nx1_sym * thick_sym / 4
        y1upper_sym = Y1_sym - Ny1_sym * thick_sym / 4

        x2lower_sym = X2_sym + Nx2_sym * thick_sym / 4
        y2lower_sym = Y2_sym + Ny2_sym * thick_sym / 4
        x2upper_sym = X2_sym - Nx2_sym * thick_sym / 4
        y2upper_sym = Y2_sym - Ny2_sym * thick_sym / 4

        # Convert symbolic expressions to numerical functions
        args = (theta_sym, thetac_sym, thick_sym, z_sym)
        func_rho1 = sp.lambdify((theta_sym, thetac_sym, z_sym), rho1_sym, "numpy")
        func_rho2 = sp.lambdify((theta_sym, thetac_sym, z_sym), rho2_sym, "numpy")

        func_x1l = sp.lambdify(args, x1lower_sym, "numpy")
        func_y1l = sp.lambdify(args, y1lower_sym, "numpy")
        func_x1u = sp.lambdify(args, x1upper_sym, "numpy")
        func_y1u = sp.lambdify(args, y1upper_sym, "numpy")

        func_x2l = sp.lambdify(args, x2lower_sym, "numpy")
        func_y2l = sp.lambdify(args, y2lower_sym, "numpy")
        func_x2u = sp.lambdify(args, x2upper_sym, "numpy")
        func_y2u = sp.lambdify(args, y2upper_sym, "numpy")

        # --- GDS Creation ---
        lib = gdstk.Library(name='SpiralLib', unit=1e-6, precision=1e-9)
        main_cell = lib.new_cell('TopSpiral')

        # Lists to store polygons for boolean operations later
        spiral_inner_polys = []
        spiral_outer_polys = []

        # Variables to store connecting points across iterations
        saveLowP0 = {}
        saveupP0 = {}
        saveLowPP = {}
        saveupPP = {}

        ix = 0
        h = np.arange(0, s * Fn + 1)

        # Geometry Generation Loop
        for ii in range(FFn, int(s * np.floor(Fn)) + 1):

            # Arrays to collect points for this segment
            X1L_list, Y1L_list = [], []
            X1U_list, Y1U_list = [], []
            X2L_list, Y2L_list = [], []
            X2U_list, Y2U_list = [], []

            for oo in range(len(freq)):
                # Angle definitions
                start_ang = iniang[0] + (2*np.pi/s)*h[ii-1]
                end_ang = iniang[0] + (2*np.pi/s)*h[ii] - GapJumps*angth

                Angle = np.arange(start_ang, end_ang + angth/1000, angth)

                Anglec1 = np.linspace(0, 2*np.pi, len(Angle))
                Anglec = np.mod(Anglec1, 2*np.pi)
                Anglec[-1] = 2*np.pi

                # Thickness and coordinate calculation
                th_val = np.abs(func_rho1(Angle, Anglec, zfixbar[oo]) -
                                func_rho2(Angle, Anglec, zfixbar[oo]))

                X1L_list.extend(func_x1l(Angle, Anglec, th_val, zfixbar[oo]))
                Y1L_list.extend(func_y1l(Angle, Anglec, th_val, zfixbar[oo]))
                X1U_list.extend(func_x1u(Angle, Anglec, th_val, zfixbar[oo]))
                Y1U_list.extend(func_y1u(Angle, Anglec, th_val, zfixbar[oo]))

                # Second set
                X2L_list.extend(func_x2l(Angle, Anglec, th_val, zfixbar[oo]))
                Y2L_list.extend(func_y2l(Angle, Anglec, th_val, zfixbar[oo]))
                X2U_list.extend(func_x2u(Angle, Anglec, th_val, zfixbar[oo]))
                Y2U_list.extend(func_y2u(Angle, Anglec, th_val, zfixbar[oo]))

            # Convert to arrays and scale to microns (1e6)
            xy1lowP0 = np.column_stack((X1L_list, Y1L_list)) * 1e6
            xy1upP0 = np.column_stack((X1U_list, Y1U_list)) * 1e6
            xy2lowPP = np.column_stack((X2L_list, Y2L_list)) * 1e6
            xy2upPP = np.column_stack((X2U_list, Y2U_list)) * 1e6

            # Save end points for stitching
            saveLowP0[ii] = xy1lowP0[-1, :]
            saveupP0[ii] = xy1upP0[-1, :]
            saveLowPP[ii] = xy2lowPP[-1, :]
            saveupPP[ii] = xy2upPP[-1, :]

            # Stitching logic
            if ii >= s + 2:
                idx_prev = ii - (s + 1)
                xy1lowP0 = np.vstack(([saveLowP0[idx_prev]], xy1lowP0))
                xy1upP0 = np.vstack(([saveupP0[idx_prev]], xy1upP0))
                xy2lowPP = np.vstack(([saveLowPP[idx_prev]], xy2lowPP))
                xy2upPP = np.vstack(([saveupPP[idx_prev]], xy2upPP))

            # Form polygons
            xy1 = np.vstack((xy1lowP0, xy1upP0[::-1]))
            xy2 = np.vstack((xy2lowPP, xy2upPP[::-1]))

            # Create GDS elements
            poly1 = gdstk.Polygon(xy1, layer=1)
            poly2 = gdstk.Polygon(xy2, layer=1)

            spiral_inner_polys.append(poly1)
            spiral_outer_polys.append(poly2)

            # Add structure references
            cell_inner = lib.new_cell(f'A{ix+1}')
            cell_inner.add(poly1)
            cell_outer = lib.new_cell(f'B{ix+1}')
            cell_outer.add(poly2)

            # --- Electrode Connection Logic (Original MATLAB translation needed for "jxy..." variables) ---
            # We must compute jxyup and jxybup because the new "Part 6" snippet depends on them
            gapAngle = np.pi / 24
            jthetaT_1 = iniang[0]
            jthickT_val = abs(func_rho1(jthetaT_1, Anglec[0], zfixbar[0]) - func_rho2(jthetaT_1, Anglec[0], zfixbar[0]))

            # --- Recalculate jxyup/bup for the Final Layer (Where ii == floor(Fn)) ---
            # This replicates the necessary data for the new snippet to work
            if ii == int(np.floor(Fn)):
                angle_end = iniang[0] + 2*np.pi*Fn
                anglec_end = 2*np.pi

                # J Set
                jx1, jy1 = func_x1u(angle_end+gapAngle/3, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+gapAngle/3, anglec_end, jthickT_val, zfixbar[0])
                jx2, jy2 = func_x1u(angle_end-gapAngle/3, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end-gapAngle/3, anglec_end, jthickT_val, zfixbar[0])
                jxyup = np.array([[jx1*1.2, jy1*1.3], [jx2*1.2, jy2*1.3]]) * 1e6

                jx3, jy3 = func_x1u(angle_end+np.pi+gapAngle/3, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+np.pi+gapAngle/3, anglec_end, jthickT_val, zfixbar[0])
                jx4, jy4 = func_x1u(angle_end+np.pi-gapAngle/3, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+np.pi-gapAngle/3, anglec_end, jthickT_val, zfixbar[0])
                jxybup = np.array([[jx3*1.2, jy3], [jx4*1.2, jy4]]) * 1e6

                # C Set (For Boolean Subtraction)
                cx1, cy1 = func_x1u(angle_end+gapAngle/2, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+gapAngle/2, anglec_end, jthickT_val, zfixbar[0])
                cx2, cy2 = func_x1u(angle_end, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end, anglec_end, jthickT_val, zfixbar[0])
                cxyup = np.array([[cx1, cy1], [cx2, cy2]]) * 1.5e6

                cx3, cy3 = func_x1u(angle_end+np.pi+gapAngle/2, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+np.pi+gapAngle/2, anglec_end, jthickT_val, zfixbar[0])
                cx4, cy4 = func_x1u(angle_end+np.pi, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+np.pi, anglec_end, jthickT_val, zfixbar[0])
                cxybup = np.column_stack(( [cx3*1.5, cx4*1.5], [cy3, cy4] )) * 1e6

                # C Set Outer (For Boolean Subtraction)
                cx1o, cy1o = func_x1u(angle_end, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end, anglec_end, jthickT_val, zfixbar[0])
                cx2o, cy2o = func_x1u(angle_end-gapAngle/2, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end-gapAngle/2, anglec_end, jthickT_val, zfixbar[0])
                cxyupo = np.array([[cx1o, cy1o], [cx2o, cy2o]]) * 1.5e6

                cx3o, cy3o = func_x1u(angle_end+np.pi, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+np.pi, anglec_end, jthickT_val, zfixbar[0])
                cx4o, cy4o = func_x1u(angle_end+np.pi-gapAngle/2, anglec_end, jthickT_val, zfixbar[0]), func_y1u(angle_end+np.pi-gapAngle/2, anglec_end, jthickT_val, zfixbar[0])
                cxybupo = np.column_stack(( [cx3o*1.5, cx4o*1.5], [cy3o, cy4o] )) * 1e6

            # --- Logic for First Iteration (ix == 0) for Boolean Ops ---
            if ix == 0:
                # C Set (Layer 3)
                cx1, cy1 = func_x1l(iniang[0], Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0], Anglec[0], jthickT_val, zfixbar[0])
                cx2, cy2 = func_x1l(iniang[0]+gapAngle/2, Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0]+gapAngle/2, Anglec[0], jthickT_val, zfixbar[0])
                cxylow = np.array([[cx1, cy1], [cx2, cy2]]) * 1e6

                cx3, cy3 = func_x1l(iniang[0]+np.pi, Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0]+np.pi, Anglec[0], jthickT_val, zfixbar[0])
                cx4, cy4 = func_x1l(iniang[0]+np.pi+gapAngle/2, Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0]+np.pi+gapAngle/2, Anglec[0], jthickT_val, zfixbar[0])
                cxyblow = np.array([[cx3, cy3], [cx4, cy4]]) * 1e6

                # C Set Outer
                cx1o, cy1o = func_x1l(iniang[0]-gapAngle/2, Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0]-gapAngle/2, Anglec[0], jthickT_val, zfixbar[0])
                cx2o, cy2o = func_x1l(iniang[0], Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0], Anglec[0], jthickT_val, zfixbar[0])
                cxylowo = np.array([[cx1o, cy1o], [cx2o, cy2o]]) * 1e6

                cx3o, cy3o = func_x1l(iniang[0]+np.pi-gapAngle/2, Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0]+np.pi-gapAngle/2, Anglec[0], jthickT_val, zfixbar[0])
                cx4o, cy4o = func_x1l(iniang[0]+np.pi, Anglec[0], jthickT_val, zfixbar[0]), func_y1l(iniang[0]+np.pi, Anglec[0], jthickT_val, zfixbar[0])
                cxyblowo = np.array([[cx3o, cy3o], [cx4o, cy4o]]) * 1e6

            ix += 1

        # --- Boolean Operations ---
        # Assemble C Electrodes (Layer 3) - Used to cut the spiral
        cxy1 = np.vstack((cxylow, cxyup))
        cxy2 = np.vstack((cxyblow, cxybup))
        cxy1o = np.vstack((cxylowo, cxyupo))
        cxy2o = np.vstack((cxyblowo, cxybupo))

        cElectrodeTop = gdstk.Polygon(cxy1, layer=3)
        cElectrodeBot = gdstk.Polygon(cxy2, layer=3)
        cElectrodeTopo = gdstk.Polygon(cxy1o, layer=3)
        cElectrodeBoto = gdstk.Polygon(cxy2o, layer=3)

        # Boolean Subtract for Spirals
        spiral_z_poly = gdstk.boolean(spiral_inner_polys, [], 'or', layer=1)
        spiral_z_final = gdstk.boolean(spiral_z_poly, [cElectrodeTop, cElectrodeBot], 'not', layer=1)

        spiral_pi_poly = gdstk.boolean(spiral_outer_polys, [], 'or', layer=1)
        spiral_pi_final = gdstk.boolean(spiral_pi_poly, [cElectrodeTopo, cElectrodeBoto], 'not', layer=1)

        jxy1 = np.vstack((jxylow, jxyup))
        jxy2 = np.vstack((jxyblow, jxybup))
        main_cell.add(gdstk.Polygon(jxy1, layer=1))
        main_cell.add(gdstk.Polygon(jxy2, layer=1))

        main_cell.add(*spiral_z_final)
        main_cell.add(*spiral_pi_final)

        # ---------------------------------------------------------
        # Part 6: Corrected Electrodes (Calculated Independently)
        # ---------------------------------------------------------
        ethick = 500
        gapAngle = np.pi/24

        # ================= RIGHT ELECTRODE =================
        # 1. Base Radius (Safe distance from spiral)
        e1_rad = np.sqrt(jxyup[0,0]**2 + jxyup[0,1]**2) * 1.1

        # 2. Angular Logic
        topAngle = iniangj_scalar + DegreeAngle
        start_angle_rad = topAngle - 5 * (np.pi/180)
        start_angle_rad_out = topAngle + 5 * (np.pi/180)

        # Stop: Strictly enforced at (90 - Gap)
        end_angle_rad = iniangj_scalar + np.pi/2 - gapAngle

        # Generate Arc Points
        t_valsin = np.linspace(start_angle_rad, end_angle_rad, 100)
        t_valsout = np.linspace(start_angle_rad_out, end_angle_rad, 100)

        # FIX: Dynamic Outer Radius (Flare)
        r_start = e1_rad + ethick
        r_end = r_start + 1000
        r_growth = np.linspace(r_start, r_end, 100)

        # Generate Coordinates
        rx_in = e1_rad * np.cos(t_valsin)
        ry_in = e1_rad * np.sin(t_valsin)

        rx_out = r_growth * np.cos(t_valsout)
        ry_out = r_growth * np.sin(t_valsout)

        # 3. Flat Top Logic (Y-Clipping)
        y_max = (e1_rad + ethick) * np.sin(end_angle_rad)
        ry_in = np.minimum(ry_in, y_max)
        ry_out = np.minimum(ry_out, y_max)

        # 4. Create RIGHT Polygon
        poly_right_pts = np.vstack((
            np.column_stack((rx_in, ry_in)),
            np.column_stack((np.flip(rx_out), np.flip(ry_out)))
        ))
        main_cell.add(gdstk.Polygon(poly_right_pts, layer=1))

        # 5. Create RIGHT Connection (Bridge)
        p_spiral_in = jxyup[0]
        p_spiral_out = jxyup[1]
        p_elec_in = np.array([rx_in[0], ry_in[0]])
        p_elec_out = np.array([rx_out[0], ry_out[0]])

        # Corrected order: Spiral_In -> Spiral_Out -> Elec_Out -> Elec_In (No Crossing)
        bridge_right = np.array([p_spiral_in, p_spiral_in, p_spiral_out, p_elec_in, p_elec_out])
        main_cell.add(gdstk.Polygon(bridge_right, layer=1))


        # ================= LEFT ELECTRODE (CALCULATED INDEPENDENTLY) =================
        # 1. Base Radius (Using Left Spiral Tips jxybup)
        e1_rad_L = np.sqrt(jxybup[0,0]**2 + jxybup[0,1]**2) * 1.1

        # 2. Angular Logic (Centered around PI)
        topAngle_L = iniangj_scalar + np.pi + DegreeAngle

        # Start angles (Offset slightly around 180 deg)
        start_angle_rad_L = topAngle_L + 5 * (np.pi/180)
        start_angle_rad_out_L = topAngle_L - 5 * (np.pi/180)

        # Stop: Strictly enforced at (90 + Gap)
        # Note: We go from ~180 DOWN to ~90
        end_angle_rad_L = iniangj_scalar + np.pi/2 + gapAngle

        # Generate Arc Points
        t_valsin_L = np.linspace(start_angle_rad_L, end_angle_rad_L, 100)
        t_valsout_L = np.linspace(start_angle_rad_out_L, end_angle_rad_L, 100)

        # Dynamic Outer Radius (Flare)
        r_start_L = e1_rad_L + ethick
        r_end_L = r_start_L + 1000
        r_growth_L = np.linspace(r_start_L, r_end_L, 100)

        # Generate Coordinates
        rx_in_L = e1_rad_L * np.cos(t_valsin_L)
        ry_in_L = e1_rad_L * np.sin(t_valsin_L)

        rx_out_L = r_growth_L * np.cos(t_valsout_L)
        ry_out_L = r_growth_L * np.sin(t_valsout_L)

        # 3. Flat Top Logic (Y-Clipping)
        # Calculate max height based on end angle
        y_max_L = (e1_rad_L + ethick) * np.sin(end_angle_rad_L)

        # Clip Y values
        ry_in_L = np.minimum(ry_in_L, y_max_L)
        ry_out_L = np.minimum(ry_out_L, y_max_L)

        # 4. Create LEFT Polygon
        poly_left_pts = np.vstack((
            np.column_stack((rx_in_L, ry_in_L)),
            np.column_stack((np.flip(rx_out_L), np.flip(ry_out_L)))
        ))
        main_cell.add(gdstk.Polygon(poly_left_pts, layer=1))

        # 5. Create LEFT Connection (Bridge)
        # Using Left Spiral Tips (jxybup)
        p_spiral_in_L = jxybup[0]   # Inner Spiral Tip
        p_spiral_out_L = jxybup[1]  # Outer Spiral Tip
        p_elec_in_L = np.array([rx_in_L[0], ry_in_L[0]])    # Inner Electrode Start
        p_elec_out_L = np.array([rx_out_L[0], ry_out_L[0]]) # Outer Electrode Start

        # Order: [Spiral_In, Spiral_Out, Elec_Out, Elec_In]
        bridge_left = np.array([p_spiral_in_L, p_spiral_out_L, p_elec_out_L, p_elec_in_L])
        main_cell.add(gdstk.Polygon(bridge_left, layer=1))

        # --- Markers and Text ---
        # Center Mark (Circle)
        r0 = 0.88 * (cWater / freq[0]) * 1e6
        # gdstk.ellipse is equivalent to the ring logic.
        # width=10 roughly maps to inner_radius = r - 5, radius = r + 5
        mark_circle = gdstk.ellipse((0, 0), radius=r0 + 5, inner_radius=r0 - 5, layer=2)
        main_cell.add(mark_circle)

        # Filename Text
        txt_pos = (-1 * rx_out[-1], -1 * np.sqrt(rx_out[-1]**2 + ry_out[-1]**2))
        main_cell.add(gdstk.Label(strv, txt_pos, layer=1))

        # Write File
        filename = f"{strv}.gds".replace('!', '')
        lib.write_gds(filename)
        print(f" > Generated: {filename}")

if __name__ == "__main__":
    run_design()

 > Generated: StitchedTorousJump-14-Freq-44.0-MHz-Cont12.gds


In [ ]:
!rm *.gds